In [1]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import Lasso
data = pd.read_csv('./HW3 附件/HW2_hr-analytics_train.csv')

In [2]:
df = data.copy()
# check nan
def check_nan(df):
    return df.isna().any().any()

def onehot_encoding(df, column_name):
    df_encoded = pd.get_dummies(df, columns=[column_name], prefix=[column_name])
    print(df.shape, "->", df_encoded.shape)
    return df_encoded

def pprint(output = '\n', show_time = False): # print and fprint at the same time
    global filename
    print(output)
    with open(filename, 'a') as f:
        if show_time:
            f.write(datetime.now().strftime("[%Y-%m-%d %H:%M:%S] "))

        f.write(str(output))
        f.write('\n')
        
def check_non_numeric_values(df, column_name):
    """
    Check if any value in the specified column is not int or float.

    Parameters:
    df (DataFrame): The DataFrame to check.
    column_name (str): The name of the column to check.

    Returns:
    bool: True if any non-numeric value is found, False otherwise.
    """
    # Get the specified column
    column = df[column_name]

    # Check if any value is not int or float
    non_numeric_values = column[~column.apply(lambda x: isinstance(x, (int, float)))]
    
    return not non_numeric_values.empty

def count_unique_values(df, column_name):
    """
    Count the number of unique values in the specified column.

    Parameters:
    df (DataFrame): The DataFrame to check.
    column_name (str): The name of the column to count unique values for.

    Returns:
    int: The count of unique values.
    """
    unique_values = df[column_name].nunique()
    return unique_values

def metrics(y_test, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Calculate precision
    precision = precision_score(y_test, y_pred)
    print("Precision:", precision)

    # Calculate recall (sensitivity)
    recall = recall_score(y_test, y_pred)
    print("Recall (Sensitivity):", recall)

    # Calculate F1 score (harmonic mean of precision and recall)
    f1 = f1_score(y_test, y_pred)
    print("F1 Score:", f1)
    # Calculate ROC AUC (Receiver Operating Characteristic Area Under the Curve)
    roc_auc = roc_auc_score(y_test, y_pred)
    print("ROC AUC:", roc_auc)
    
col_not_num = []
for col in df.columns:
    if check_non_numeric_values(df, col):
        col_not_num.append(col)

print(f"Doing onehot encoding on col: {col_not_num}")
for col in col_not_num:
    df = onehot_encoding(df, col)


Doing onehot encoding on col: ['sales', 'salary']
(10000, 10) -> (10000, 19)
(10000, 19) -> (10000, 21)


In [3]:
target_column = 'left'
test_size = 0.2
random_state = 42

trans = True
if trans: # 956 -> 957
    scaler = StandardScaler()
    df['average_montly_hours'] = scaler.fit_transform(df[['average_montly_hours']])
    # df['time_spend_company'] = scaler.fit_transform(df[['time_spend_company']])



X = df.drop(columns=[target_column])
y = df[target_column]

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size, random_state=random_state)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=random_state)

In [4]:


# ensemble_classifier.fit(X_train, y_train)
# y_pred = ensemble_classifier.predict(X_train)
# accuracy = accuracy_score(y_train, y_pred)
# print("Improved Accuracy:", accuracy)
# confusion = confusion_matrix(y_train, y_pred)
# print(confusion)
# metrics(y_train, y_pred)

# y_pred = ensemble_classifier.predict(X_val)
# accuracy = accuracy_score(y_val, y_pred)
# print("Improved Accuracy:", accuracy)
# confusion = confusion_matrix(y_val, y_pred)
# print(confusion)
# metrics(y_val, y_pred)

from sklearn.tree import DecisionTreeClassifier

# Create a Decision Tree classifier
decision_tree_classifier = DecisionTreeClassifier()

# Fit the Decision Tree classifier to the training data
decision_tree_classifier.fit(X_train, y_train)

# Predict on the training data and calculate accuracy
y_pred = decision_tree_classifier.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("Training Accuracy:", accuracy)
confusion = confusion_matrix(y_train, y_pred)
print(confusion)
metrics(y_train, y_pred)

# Predict on the validation data and calculate accuracy
y_pred = decision_tree_classifier.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)
confusion = confusion_matrix(y_val, y_pred)
print(confusion)
metrics(y_val, y_pred)


Training Accuracy: 1.0
[[6090    0]
 [   0 1910]]
Accuracy: 1.0
Precision: 1.0
Recall (Sensitivity): 1.0
F1 Score: 1.0
ROC AUC: 1.0
Validation Accuracy: 0.976
[[746  13]
 [ 11 230]]
Accuracy: 0.976
Precision: 0.9465020576131687
Recall (Sensitivity): 0.9543568464730291
F1 Score: 0.9504132231404959
ROC AUC: 0.968614523368267


In [5]:
test_data = pd.read_csv('./HW3 附件/HW2_hr-analytics_test.csv')
test_data.shape

(5000, 9)

In [6]:
test_df = test_data.copy()
col_not_num = []
for col in test_df.columns:
    if check_non_numeric_values(test_df, col):
        col_not_num.append(col)

print(f"Doing onehot encoding on col: {col_not_num}")
for col in col_not_num:
    test_df = onehot_encoding(test_df, col)

Doing onehot encoding on col: ['sales', 'salary']
(5000, 9) -> (5000, 18)
(5000, 18) -> (5000, 20)


In [8]:
test_size = 0.2
random_state = 42

trans = True
if trans: # 956 -> 957
    scaler = StandardScaler()
    test_df['average_montly_hours'] = scaler.fit_transform(test_df[['average_montly_hours']])

X = test_df
y_real_pred = decision_tree_classifier.predict(X)
y_real_pred.shape

(5000,)

In [9]:
result = y_real_pred.reshape(-1, 1)  # Reshape to (1000, 1)
# Create a DataFrame with one column 'left'
result_df = pd.DataFrame(data=result, columns=['left'])
print(result_df)
result_df.to_csv('HW2_hr-analytics_test_sol.csv', index=False)

      left
0        0
1        0
2        1
3        1
4        0
...    ...
4995     0
4996     0
4997     0
4998     0
4999     1

[5000 rows x 1 columns]


In [10]:
print(X['average_montly_hours'])

0      -0.827940
1      -0.927944
2       0.352100
3       1.272131
4      -1.307957
          ...   
4995    0.012088
4996   -1.487963
4997    1.132127
4998   -0.427927
4999   -0.867942
Name: average_montly_hours, Length: 5000, dtype: float64
